# 进阶题一：扩散模型开山之作——DDPM论文精读
***

1. 在DDPM出现之前，主流的生成模型（如GAN, VAE）各有什么优缺点？DDPM的提出主要解决了哪一类问题或实现了怎样的提升？

+ 在DDPM出现之前，GAN存在训练不稳定，缺乏概率解释的问题，VAE由于重构损失通常是像素级的均方误差存在生成样本模糊的问题。
+ DDPM 的提出，主要解决了GAN的训练不稳定和模式崩塌问题，同时克服了VAE图像模糊的问题，同时实现了稳定训练和高质量生成。


2. 请用自己的话简要描述DDPM的两个核心过程：前向过程 (Forward Process)和反向过程 (Denoising Process)

+ 前向过程：前向过程是一个向真实数据中逐步加入高斯噪声，最后变成一个接近于标准正态分布的纯噪声的过程。
+ 反向过程：反向过程是一个通过预测前向过程中对应时间步的噪声将纯噪声逐步去噪最后得到清晰的图像的过程。

3. 前向过程
    1. 前向过程的每一步是如何定义的？为什么说这是一个固定的、无需学习的马尔可夫过程？
    
    + 前向过程的每一步由条件高斯分布$q(x_t \mid x_{t-1}) = \mathcal{N}\!\Big(x_t;\,\sqrt{1-\beta_t}\,x_{t-1},\,\beta_t I\Big)$定义，它描述了在上一步数据$x_{t-1}$的基础上加入高斯噪声后得到$x_t$的过程。这里$\beta_t$是一个随时间步t变化的噪声强度参数，通常设计为随t增大而逐渐增大。
    + 由于$\{\beta_t\}$是人为设定的超参数，而不是通过训练学习得到的，所以整个前向过程是固定的、无需学习的。同时，每一步的条件分布只依赖于前一步$x_{t-1}$，因此构成一个马尔可夫过程。

    2. 该过程有一个极其重要的特性：可以直接从初始图像$x_0$得到任意时刻$t$的加噪图像$\mathbf{x}_t$。请写出这个采样公式，并解释为什么这个特性对于高效训练模型至关重要。

    + $x_t = \sqrt{\bar{\alpha}_t}\,x_0 + \sqrt{1-\bar{\alpha}_t}\, \epsilon \quad \text{for} \quad \epsilon \sim \mathcal{N}\!\Big(0 \,\ I\Big) \quad \text{where} \quad \alpha_t = 1 - \beta_t \quad \text{and} \quad \bar{\alpha}_t = \prod_{s = 1}^t \alpha_s$
    + 这样做的好处一是可以避免逐步采样，支持一步直接采样 $x_t$ ；二是能在任意时刻随机取样，而不需要真的跑完整个扩散链，使得训练可以并行化、随机化，效率极高；三是可以通过此公式将预测目标转化为预测噪声$\epsilon$


4. 反向过程
   1. 在实践中，模型（通常是一个U-Net）被训练来预测什么？为什么这样预测？
   + 模型被训练用来预测对应时间步在前向过程中加入的噪声。
   + 这样预测的原因一是由上述前向过程的采样公式中 $x_t$ , $x_0$ 和 $\epsilon$ 的关系，使我们可以通过 $\epsilon$ 和 $x_0$ 还原 $x_t$；二是因为噪声 $\epsilon$ 来自标准正态分布，分布简单、固定，使训练更稳定；三是噪声始终符合正态分布，这使得在不同时间步上噪声呈现不同强度时，能学到相似的“去噪”能力。
   2. 数学推导反向过程的公式
   + 前向过程相关推导
   $$
   q(x_t \mid x_{t-1}) = \mathcal{N}\!\big(x_t;\,\sqrt{\alpha_t}\,x_{t-1},\,\beta_t I\big),\quad \alpha_t=1-\beta_t
   $$

   $$
   q(x_t \mid x_0) = \mathcal{N}\!\big(x_t;\,\sqrt{\bar{\alpha}_t}\,x_0,\,(1-\bar{\alpha}_t)I\big),\quad
   \bar{\alpha}_t=\prod_{i=1}^t \alpha_i
   $$

   $$
   q(x_{t-1}\mid x_t,x_0)=\mathcal{N}\!\big(x_{t-1};\,\tilde{\mu}_t(x_t,x_0),\,\tilde{\beta}_t I\big)
   $$

   $$
   \tilde{\beta}_t=\frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t}\,\beta_t,\quad
   \tilde{\mu}_t(x_t,x_0)=\frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{1-\bar{\alpha}_t}\,x_0
   +\frac{\sqrt{\alpha_t}\,(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}\,x_t
   $$
   + 反向过程：反向过程的目标是为了学习一个马尔科夫链去拟合真实的后验分布。

      我们希望学习一个反向马尔可夫链：
   $$
   p_\theta(x_{t-1}\mid x_t) = \mathcal{N}\!\big(x_{t-1};\,\mu_\theta(x_t,t),\,\Sigma_\theta(x_t,t)\big)
   $$
   
      真实的后验分布是：
   $$
   q(x_{t-1}\mid x_t,x_0) = \frac{q(x_t\mid x_{t-1})\,q(x_{t-1}\mid x_0)}{q(x_t\mid x_0)}
   $$
   + 由上述前向过程的公式，$x_0$可以改写为
   $$
   x_0 = \frac{1}{\sqrt{\bar{\alpha}_t}}\Big(x_t - \sqrt{1-\bar{\alpha}_t}\,\epsilon\Big)
   $$
   + 因此预测目标可以改写为
   $$
   \hat{x}_0(x_t,t)=\frac{1}{\sqrt{\bar{\alpha}_t}}\Big(x_t-\sqrt{1-\bar{\alpha}_t}\,\epsilon_\theta(x_t,t)\Big)
   $$
   + 将 $\hat{x}_0$ 代入 $\tilde{\mu}_t$，并化简，得到原论文中的反向均值：
   $$
   \mu_\theta(x_t,t)=\frac{1}{\sqrt{\alpha_t}}\left(x_t-\frac{\beta_t}{\sqrt{1-\bar{\alpha}_t}}\,\epsilon_\theta(x_t,t)\right)
   $$
   + 因此最后的采样公式为
   $$
   x_{t-1}\sim \mathcal{N}\!\big(\mu_\theta(x_t,t),\,\tilde{\beta}_t I\big)
   $$
   + 关于方差的说明:
     - 真实后验方差: $\tilde{\beta}_t=\frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t}\beta_t$。
     - 原论文设置（常用）: 反向方差 $\Sigma_\theta$ 可固定为某个预设（如 $\beta_t I$），以简化训练与采样稳定性。
     - 改进： 将 $\log\Sigma_\theta$ 学习为介于 $\log\beta_t$ 与 $\log\tilde{\beta}_t$ 的插值，以兼顾逼近真实后验与数值稳定。这与“数据分布离散/集中”无直接选择准则，而是训练目标与稳定性上的折中。


5. DDPM的损失函数是怎么设计的？
+ 原始预测目标是一个变分下界(ELBO)
$$
\log p_\theta(x_0) \geq \mathbb{E}_q\Bigg[ \log \frac{p_\theta(x_{0:T})}{q(x_{1:T}\mid x_0)} \Bigg]
$$
+ 展开整理为 $T$ 项KL散度，其中 $L_T$ 和 $L_0$是常数项，关键项是中间项。 
$$
\mathcal{L}
= \mathbb{E}_q \Bigg[
\underbrace{D_{\mathrm{KL}}\!\big(q(x_T \mid x_0)\,\|\,p(x_T)\big)}_{L_T}
+ \sum_{t>1} \underbrace{D_{\mathrm{KL}}\!\big(q(x_{t-1}\mid x_t,x_0)\,\|\,p_\theta(x_{t-1}\mid x_t)\big)}_{L_{t-1}}
- \underbrace{\log p_\theta(x_0 \mid x_1)}_{L_0}
\Bigg]
$$
+ 原论文中通过前向和反向过程都是高斯分布的特性将关键项进一步化简为
$$
L_{t-1} = \mathbb{E}_q \left[ \frac{1}{2\sigma_t^2} 
\left\| \tilde{\mu}_t(\mathbf{x}_t, \mathbf{x}_0) - \mu_\theta(\mathbf{x}_t, t) \right\|^2 \right] + C
$$
+ 去掉常数项并将模型简化为预测噪声后，通过上述反向过程中的相关推导进一步简化为
$$
\mathbb{E}_{x_0, \epsilon} \left[ 
\frac{\beta_t^2}{2\sigma_t^2 \alpha_t (1 - \bar{\alpha}_t)} 
\left\| \epsilon - \epsilon_\theta \!\left( \sqrt{\bar{\alpha}_t}\, x_0 + \sqrt{1 - \bar{\alpha}_t}\,\epsilon,\, t \right) \right\|^2 
\right]
$$

6. 本论文以及后续的大多数扩散模型都选择U-Net作为核心的神经网络架构。你认为U-Net架构有什么特性使其特别适合于在扩散模型中执行“去噪”这个任务？
+ U-Net 最初被提出用于医学图像分割任务，其网络结构由对称的编码器（收缩路径）和解码器（扩张路径）组成。编码器通过逐层下采样逐渐提取更高层次的语义特征；与此同时，每一层的特征图通过跳跃连接直接传递至对应的解码器层，以保留空间细节信息。在解码器部分，特征图逐步上采样以恢复空间分辨率，并与来自编码器的对应层特征融合，从而在重建过程中兼顾全局语义与局部细节。这一结构特性很好地满足了去噪任务中对全局一致性与局部精细度的双重需求。由于噪声往往破坏局部细节，跳跃连接在去噪过程中能够有效帮助模型保留并恢复这些信息。此外，U-Net 还可以在纵向结构中引入时间步嵌入，从而适应不同噪声水平下的去噪过程。

7. DDPM的主要缺点是什么？
+ DDPM 的主要缺点是：采样速度极慢（通常需要数百上千步），推理和训练成本高（因为采样过程需要很多步），生成质量与速度难以兼顾，方差固定（原文中作者承认他们没有让模型学习方差，而是直接固定为两种可能的选择
）。

8. 阅读DDPM论文后，你是否了解后续的研究（如DDIM, Stable Diffusion）是如何尝试解决或缓解上述局限性的？请简要描述至少一种改进思路。
+ DDIM加速了采样过程，改进的思路是通过将反向过程改为非马尔科夫的确定性采样，允许“跳步”生成，从而在保持高质量的同时显著加快采样速度。在DDIM中，作者构造了一个确定性映射：
$$
x_{t-1} = \sqrt{\bar{\alpha}_{t-1}}\,\hat{x}_0(x_t,t) + \sqrt{1-\bar{\alpha}_{t-1}}\,\epsilon_\theta(x_t,t)
$$
其中 $\hat{x}_0$ 是通过预测噪声 $\epsilon_\theta$ 反推出的原始图像估计。这个公式类似于DDPM中的前向过程。
在 DDIM 中，作者发现训练目标只依赖于边缘分布 $q(x_t|x_0)$ ，因此采样时不必严格走完所有步。
在DDPM的很多步中，变化是极其微小的（因为 $\beta_t$ 很小），DDIM 通过非马尔科夫化直接跳过这些冗余步骤。


9. 尝试使用Pytorch复现DDPM（不强制要求从零开始进行完整复现）
本代码复现是基于本题的相关资料中的教程实现的，代码和原教程基本无异，主要是为了理解每一步在做什么。

In [6]:
%pip install -q -U einops

Note: you may need to restart the kernel to use updated packages.


In [1]:
import math
from inspect import isfunction
from functools import partial

%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from einops import rearrange, reduce
from einops.layers.torch import Rearrange

import torch
from torch import nn, einsum
import torch.nn.functional as F

d:\anaconda3\envs\Tpytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+ 定义一些辅助的功能函数

In [2]:
# 判断变量是否存在
def exists(x):
    return x is not None

# 如果val存在就返回它，否则返回默认值d。这里也可以返回函数
def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

# 把一个整数 num 拆分成若干个 divisor 大小的组，最后一组可能小于 divisor
# 常用于把 batch 拆分成小块，或者在采样时分组处理。
def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr


#实现残差连接
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        return self.fn(x, *args, **kwargs) + x


# U-Net中的向上取样函数
def Upsample(dim, dim_out=None):
    return nn.Sequential(
        nn.Upsample(scale_factor=2, mode="nearest"),
        nn.Conv2d(dim, default(dim_out, dim), 3, padding=1),
    )


# U-Net中的向下取样函数
def Downsample(dim, dim_out=None):
    # No More Strided Convolutions or Pooling
    return nn.Sequential(
        Rearrange("b c (h p1) (w p2) -> b (c p1 p2) h w", p1=2, p2=2),
        nn.Conv2d(dim * 4, default(dim_out, dim), 1),
    )

+ Position embedding，用于注入位置（时间步）信息

In [3]:
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

+ 替换标准卷积层，构建残差块

In [4]:
class WeightStandardizedConv2d(nn.Conv2d):
    """
    https://arxiv.org/abs/1903.10520
    weight standardization purportedly works synergistically with group normalization
    """

    def forward(self, x):
        eps = 1e-5 if x.dtype == torch.float32 else 1e-3

        weight = self.weight
        mean = reduce(weight, "o ... -> o 1 1 1", "mean")
        var = reduce(weight, "o ... -> o 1 1 1", partial(torch.var, unbiased=False))
        normalized_weight = (weight - mean) * (var + eps).rsqrt()

        return F.conv2d(
            x,
            normalized_weight,
            self.bias,
            self.stride,
            self.padding,
            self.dilation,
            self.groups,
        )


class Block(nn.Module):
    def __init__(self, dim, dim_out, groups=8):
        super().__init__()
        self.proj = WeightStandardizedConv2d(dim, dim_out, 3, padding=1)
        self.norm = nn.GroupNorm(groups, dim_out)
        self.act = nn.SiLU()

    def forward(self, x, scale_shift=None):
        x = self.proj(x)
        x = self.norm(x)

        if exists(scale_shift):
            scale, shift = scale_shift
            x = x * (scale + 1) + shift

        x = self.act(x)
        return x


class ResnetBlock(nn.Module):
    """https://arxiv.org/abs/1512.03385"""

    def __init__(self, dim, dim_out, *, time_emb_dim=None, groups=8):
        super().__init__()
        self.mlp = (
            nn.Sequential(nn.SiLU(), nn.Linear(time_emb_dim, dim_out * 2))
            if exists(time_emb_dim)
            else None
        )

        self.block1 = Block(dim, dim_out, groups=groups)
        self.block2 = Block(dim_out, dim_out, groups=groups)
        self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb=None):
        scale_shift = None
        if exists(self.mlp) and exists(time_emb):
            time_emb = self.mlp(time_emb)
            time_emb = rearrange(time_emb, "b c -> b c 1 1")
            scale_shift = time_emb.chunk(2, dim=1)

        h = self.block1(x, scale_shift=scale_shift)
        h = self.block2(h)
        return h + self.res_conv(x)


+ 加入注意力机制

In [5]:
class Attention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)
        self.to_out = nn.Conv2d(hidden_dim, dim, 1)

    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k, v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )
        q = q * self.scale

        sim = einsum("b h d i, b h d j -> b h i j", q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        out = einsum("b h i j, b h d j -> b h i d", attn, v)
        out = rearrange(out, "b h (x y) d -> b (h d) x y", x=h, y=w)
        return self.to_out(out)

class LinearAttention(nn.Module):
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        hidden_dim = dim_head * heads
        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)

        self.to_out = nn.Sequential(nn.Conv2d(hidden_dim, dim, 1), 
                                    nn.GroupNorm(1, dim))

    def forward(self, x):
        b, c, h, w = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=1)
        q, k, v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )

        q = q.softmax(dim=-2)
        k = k.softmax(dim=-1)

        q = q * self.scale
        context = torch.einsum("b h d n, b h e n -> b h d e", k, v)

        out = torch.einsum("b h d e, b h d n -> b h e n", context, q)
        out = rearrange(out, "b h c (x y) -> b (h c) x y", h=self.heads, x=h, y=w)
        return self.to_out(out)
